In [33]:
import pyscheduling.SMSP.risijwiCi as sm
import pyscheduling.SMSP.riwiCi as sm_ci
import pyscheduling.SMSP.SM_Methods as sm_methods

%load_ext autoreload
%load_ext pycodestyle_magic
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The pycodestyle_magic extension is already loaded. To reload it, use:
  %reload_ext pycodestyle_magic


## New problem risijwiCi

In [34]:
instance = sm.risijwiCi_Instance.generate_random(40, Wmax=2)

In [28]:
results_dict = {}
for rule_number in range(4):
    obj = sm.Heuristics.list_heuristic(instance, rule_number, False).best_solution.objective_value
    obj_reverse = sm.Heuristics.list_heuristic(instance, rule_number, True).best_solution.objective_value
    
    results_dict[f'Rule_{rule_number}'] = obj
    results_dict[f'Rule_{rule_number}-reverse'] = obj_reverse 
    
print(results_dict)

{'Rule_0': 137458, 'Rule_0-reverse': 173867, 'Rule_1': 202864, 'Rule_1-reverse': 111767, 'Rule_2': 196509, 'Rule_2-reverse': 109573, 'Rule_3': 184029, 'Rule_3-reverse': 125023}


In [40]:
solve_result_grasp = sm_methods.Heuristics.grasp(instance, 0, 0.2, 5)

In [41]:
solve_result_biba = sm_methods.Heuristics.BIBA(instance)

In [42]:
solve_result_list = sm.Heuristics.list_heuristic(instance, rule_number=1)

In [53]:
solve_result_lahc = sm_methods.Metaheuristics.lahc(instance, **{"time_limit_factor":0.2})

In [54]:
solve_result_sa = sm_methods.Metaheuristics.SA(instance, **{"time_limit_factor":0.2})

In [55]:
print(solve_result_lahc.best_solution)

Objective : 82027
Job_schedule (job_id , start_time , completion_time) | objective
(14, 0, 13) : (30, 33, 43) : (21, 65, 78) : (31, 105, 119) : (25, 147, 165) : (20, 201, 232) : (17, 255, 266) : (10, 300, 333) : (7, 358, 368) : (24, 391, 428) : (12, 496, 529) : (5, 549, 559) : (13, 581, 601) : (8, 645, 693) : (16, 772, 811) : (36, 850, 862) : (6, 897, 941) : (9, 983, 997) : (27, 1035, 1081) : (22, 1152, 1186) : (4, 1257, 1298) : (3, 1327, 1338) : (18, 1369, 1390) : (11, 1439, 1482) : (39, 1566, 1608) : (33, 1698, 1746) : (37, 1822, 1858) : (0, 1905, 1923) : (26, 1965, 2001) : (2, 2051, 2074) : (35, 2128, 2176) : (34, 2262, 2305) : (29, 2381, 2412) : (32, 2454, 2474) : (19, 2519, 2560) : (1, 2642, 2686) : (15, 2769, 2807) : (38, 2881, 2917) : (23, 2989, 3026) : (28, 3109, 3153) | 82027


## Old problem riwiCi

In [17]:
instance = sm_ci.riwiCi_Instance.generate_random(10)

In [18]:
solve_result = sm_methods.Heuristics_Cmax.grasp(instance, 0.5, 0.5, 5)

In [19]:
%%timeit
solve_result_1 = sm_ci.Heuristics.WSAPT(instance)

In [40]:
%%timeit
rule = lambda instance, job_id : float(instance.W[job_id])/float(instance.P[job_id])
filter_fun = lambda instance, ci, job_id : instance.R[job_id]<=ci
solve_result_2 = sm_methods.Heuristics.dynamic_dispatch_rule(instance,rule, filter_fun, reverse=True)

84.6 µs ± 910 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [31]:
print(solve_result_1)
print(solve_result_2)

Search stopped with status : FEASIBLE
 Solution is : 
 Objective : 2055
Job_schedule (job_id , start_time , completion_time) | objective
(6, 0, 2) : (0, 7, 66) : (5, 66, 88) : (7, 88, 113) : (9, 113, 152) : (1, 152, 195) : (8, 195, 255) : (2, 255, 317) : (4, 317, 387) : (3, 387, 480) | 2055 
Runtime is : 6.069301161915064e-05s 
time to best is : -1s 

Search stopped with status : FEASIBLE
 Solution is : 
 Objective : 2055
Job_schedule (job_id , start_time , completion_time) | objective
(6, 0, 2) : (0, 7, 66) : (5, 66, 88) : (7, 88, 113) : (9, 113, 152) : (1, 152, 195) : (8, 195, 255) : (2, 255, 317) : (4, 317, 387) : (3, 387, 480) | 2055 
Runtime is : 0.0002623129985295236s 
time to best is : -1s 

